# Abrindo as Imagens da Base LIDC-IDRI

Nesta primeira etapa do projeto, nós utilizamos as bibliotecas Pydicom e Numpy para abrir as imagens e gerar arquivos em um formato que será utilizado na próxima etapa. 

A biblioteca Pydicom é a responsável por abrir os arquivos originais na base, no formato dicom (.dcm). Posteriormente, são coletadas desses arquivos o array de pixels e informações presentes no cabeçalho que são relevantes a análise que será realizada posteriormente. 

Esses dados coletados são salvos em um array da biblioteca Numpy no formato (.npy).

Para mais informações sobre o projeto, leia o README deste repositório.

## Importação das Bibliotecas

As bibliotecas utilizadas nessa etapa do projeto foram:

* pydicom: Responsável pela manipulação das imagens no formato dicom
* glob: Responsável por listar os nomes dos arquivos dentro de um padrão
* os: Responsável por operações de sistema operacional
* numpy: Operações matemáticas e criação de arrays

Essas libs são importadas no código abaixo:

In [ ]:
import pydicom 
from glob import glob 
import os, os.path 
import numpy as np 

## Configurando as fontes de dados

In [ ]:
image_directory = "/media/acucena/Arquivos/Base ICV/LIDC-IDRI/img/teste/*" # Entrada
output_path = "/media/acucena/Arquivos/Base ICV/NPY/" # Saída
path_to_images = glob(image_directory) # Lista de arquivos presentes no diretório de entrada

## Processamento

O código abaixo irá iterar por todos os exames da base e executando as seguintes ações:

1. Navega pelos arquivos dos diretórios dos exames encontrando a quantidade de arquivos nessas pastas. Nós estamos interessados no diretório com o maior número de arquivos (Um exame consta de alguns subníveis de arquivo e alguns possuem além do exame de TC, um exame de raio-X). 

2. Seleciona a pasta com o maior número de arquivos e cria uma lista os arquivos presentes na mesma. 

3. Itera sobre os arquivos da lista gerada no passo 2:
  * Lê os arquivos (cada arquivo desse é chamado de slice)
  * Coleta alguns dados do arquivo (o arquivo dicom é composto por um array de pixels e um cabeçalho com informações relevantes sobre a imagem e o paciente relativo àquele exame).
  
  
4. Salva os arrays com os dados selecionados em arquivos no formato .npy.
  
    

In [ ]:
largest_stack = 0

for patient in path_to_images:
    
    # Passo 1
    subdirs = glob(patient + "/*")
    # 1st Branch
    first_subdir = glob(subdirs[0]+"/*")[0]
    first_len = len([name for name in os.listdir(first_subdir + "/")])
    # 2nd Branch
    second_subdir = glob(subdirs[-1] + "/*")[0]
    second_len = len([name for name in os.listdir(second_subdir + "/")])

    # Passo 2
    # If 1st longer then build structure from 1st data
    # If 2nd longer then build structure from 2nd data
    if first_len > second_len:
        patient_image_folder = first_subdir + "/*.dcm"
    else:
        patient_image_folder = second_subdir + "/*.dcm"
    
    # Passo 3
    image_stack = []
    cur_stack_size = 0
    for image in glob(patient_image_folder):
        # Read in each image and construct 3d arrays
        ds = pydicom.dcmread(image)
        image_stack.append([ds.pixel_array, ds.SliceLocation, ds.InstanceNumber, 
                            ds.RescaleIntercept, ds.RescaleSlope])
        cur_stack_size = cur_stack_size + 1
        
    # Keep track of largest num of slices for zero pad
    if cur_stack_size > largest_stack:
        largest_stack = cur_stack_size
    patient_array = np.asarray(image_stack)
    
    # Passo 4
    id = patient.split("/")[8]
    np.save(output_path + "%s.npy" % (id), patient_array)
    print("Patient: " + id + " converted and saved to npy")

## Conclusão

Após essa etapa, passaremos para o pré-processamento e a segmentação do parênquima e dos nódulos.